# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [ ]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug

In [ ]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [ ]:
# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

dogImagesFolder = 's3://sagemaker-us-east-1-155306683617/dogImages'

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [ ]:
hyperparameter_ranges = {
    'lr': ContinuousParameter(0.0001, 0.25),
    'batch-size': CategoricalParameter([32, 64, 128, 256, 512]),
    'hidden-units': CategoricalParameter([128, 150, 192, 256]),
    'dropout': CategoricalParameter([0.0, 0.2, 0.4, 0.6])
}

objective_metric_name = "Average test set loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Average test set loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [ ]:
role = sagemaker.get_execution_role()

estimator = PyTorch(
    entry_point='train_model.py',
    role=role,
    py_version='py39',
    framework_version="1.8",
    instance_count=1,
    # instance_type='ml.g4dn.xlarge'   # cheapest GPU + 4 vcpu / 16GB RAM
    instance_type="ml.m5.large" # CPU 2 core / 8GB RAM
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=4,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [ ]:
datapaths = {
    'train': dogImagesFolder + '/train',       
    'test': dogImagesFolder + '/test',      
    'val': dogImagesFolder + '/valid'
}

tuner.fit(datapaths, wait=True)

In [ ]:
best_estimator = tuner.best_estimator()
best_training_job = tuner.best_training_job()

#Get the hyperparameters of the best trained model
best_hyperparams = best_estimator.hyperparameters()

print(best_estimator)
print(best_training_job)
print(best_hyperparams)

## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [ ]:
# TODO: Set up debugging and profiling rules and hooks

In [ ]:
# TODO: Create and fit an estimator

estimator = # TODO: Your estimator here

In [ ]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [ ]:
# TODO: Display the profiler output

## Model Deploying

In [ ]:
# TODO: Deploy your model to an endpoint

predictor=estimator.deploy() # TODO: Add your deployment configuration like instance type and number of instances

In [ ]:
# TODO: Run an prediction on the endpoint

image = # TODO: Your code to load and preprocess image to send to endpoint for prediction
response = predictor.predict(image)

In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()